In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

root = '/root/data/SVHN/'

# Load the SVHN
train_dataset = torchvision.datasets.SVHN(root=root, split='train', transform=transform, download=False)
test_dataset = torchvision.datasets.SVHN(root=root, split='test', transform=transform, download=False)

bs = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=True)

In [2]:
class CVAE(nn.Module):

    def __init__(self, condi_dim, channels, latent_dim) -> None:
        super().__init__()

        # encoder
        pre_channel = 3
        modules = []
        img_length = 32
        
        for i in range(len(channels)-1):
            modules.append(
                nn.Sequential(
                    nn.Conv2d(pre_channel,
                              channels[i+1],
                              kernel_size=3,
                              stride=2,
                              padding=1),
                    nn.BatchNorm2d(channels[i+1]),
                    nn.ReLU()
                )
            )
            pre_channel = channels[i+1]
            img_length = (img_length-1)//2+1
        
        self.encoder_projection = nn.Sequential(
                nn.Linear(pre_channel * img_length * img_length + condi_dim, pre_channel * img_length * img_length),
                nn.ReLU()
        )
        
        self.encoder = nn.Sequential(*modules)
        self.mean_linear = nn.Linear(pre_channel * img_length * img_length,
                                     latent_dim)
        self.var_linear = nn.Linear(pre_channel * img_length * img_length,
                                    latent_dim)
        self.latent_dim = latent_dim
        
        # decoder
        modules = []
        self.decoder_projection = nn.Linear(
            latent_dim + condi_dim, pre_channel * img_length * img_length)
        self.decoder_input_chw = (pre_channel, img_length, img_length)

        for i in range(len(channels)-1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(channels[len(channels)-i-1],
                                       channels[len(channels)-i-2],
                                       kernel_size=3,
                                       stride=2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(channels[len(channels)-i-2]),
                    nn.ReLU()
                )
            )
        self.decoder_layers = nn.Sequential(*modules)
        
    def decoder(self, z, c):
        z = torch.cat([z, c], dim = 1)
        z = self.decoder_projection(z)
        z = torch.reshape(z, (-1, *self.decoder_input_chw))
        decoded = self.decoder_layers(z)
        return decoded

    def forward(self, x, c):
        x = self.encoder(x)
        x = torch.flatten(x, 1)
        x = torch.cat([x, c], dim = 1)
        encoded = self.encoder_projection(x)
        mean = self.mean_linear(encoded)
        logvar = self.var_linear(encoded)
        eps = torch.randn_like(logvar)
        std = torch.exp(logvar / 2)
        z = eps * std + mean
        decoded = self.decoder(z, c)
        return decoded, mean, logvar

In [3]:
from time import time
import numpy as np

n_epochs = 1000
num_class = 10
learning_rate = 1e-2
kl_weight = 5e-7

def one_hot_encode(labels, num_classes):
    one_hot_labels = np.zeros((len(labels), num_classes))
    one_hot_labels[np.arange(len(labels)), labels] = 1
    return one_hot_labels.tolist()

def elbo_loss(x, x_hat, mean, logvar):
    recons_loss = F.mse_loss(x_hat, x)
    kl_loss = -0.5 * torch.sum(1 + logvar - mean**2 - torch.exp(logvar))
    loss = (recons_loss + kl_loss * kl_weight)/len(x)
    return loss

def reconst_loss(x, x_hat):
    return F.mse_loss(x_hat, x) / len(x)

def kl_loss(mean, logvar):
    return -0.5 * torch.sum(1 + logvar - mean**2 - torch.exp(logvar)) / len(mean)

def train(device, model):
    optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
    begin_time = time()
    # train
    with open('./loss1.txt', 'w') as file:
        for i in range(n_epochs):
            for batch_idx, (x, label) in enumerate(train_loader):
                x = x.to(device)
                label = one_hot_encode(label, num_classes = 10)
                label = torch.tensor(label)
                label = label.to(device)
                x_hat, mean, logvar = model(x, label)
                loss = elbo_loss(x, x_hat, mean, logvar)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            # estimate loss
            model.eval()
            with torch.no_grad():
                each_epoch = 10
                n_samples = 100
                #train
                indices = torch.randperm(len(train_dataset))[:n_samples]
                x = torch.stack([train_dataset[i][0] for i in indices]).to(device)
                label = one_hot_encode([train_dataset[i][1] for i in indices], num_classes = 10)
                label = torch.tensor(label).to(device)
                x_hat, mean, logvar = model(x, label)
                loss = elbo_loss(x, x_hat, mean, logvar)
                loss_recons = reconst_loss(x, x_hat)
                loss_kl = kl_loss(mean, logvar)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.7f}'.format(i, loss))
                    print('====> Epoch: {} reconst loss: {:.7f}'.format(i, loss_recons))
                    print('====> Epoch: {} kl loss: {:.7f}'.format(i, loss_kl))
                file.write(str(loss.item()) + ' ' + str(loss_recons.item()) + ' ' + str(loss_kl.item()) + ' ')
                #test
                indices = torch.randperm(len(test_dataset))[:n_samples]
                x = torch.stack([test_dataset[i][0] for i in indices]).to(device)
                label = one_hot_encode([test_dataset[i][1] for i in indices], num_classes = 10)
                label = torch.tensor(label).to(device)
                x_hat, mean, logvar = model(x, label)
                loss = elbo_loss(x, x_hat, mean, logvar)
                loss_recons = reconst_loss(x, x_hat)
                loss_kl = kl_loss(mean, logvar)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.7f}'.format(i, loss))
                    print('====> Epoch: {} reconst loss: {:.7f}'.format(i, loss_recons))
                    print('====> Epoch: {} kl loss: {:.7f}'.format(i, loss_kl))
                file.write(str(loss.item()) + ' ' + str(loss_recons.item()) + ' ' + str(loss_kl.item()) + '\n')
            
            #time
            if(i % each_epoch == 0):
                training_time = time() - begin_time
                minute = int(training_time // 60)
                second = int(training_time % 60)
                print(f'time loss {minute}:{second}')
        
        torch.save(model, './vae_model1.pth')
        
    tot_training_time = time() - begin_time
    minute = int(tot_training_time // 60)
    second = int(tot_training_time % 60)
    print(f'total time loss {minute}:{second}')

def initialize_parameters(model):
    for param in model.parameters():
        param.data.normal_(mean=0, std=0.01)

In [ ]:
def main():
    device = 'cuda:0'

    model = CVAE(condi_dim = num_class, channels = [3, 500, 500, 500, 500], latent_dim = 20).to(device)
    initialize_parameters(model)

    train(device, model)

if __name__ == '__main__':
    main()

====> Epoch: 0 elbo loss: 0.0005348
====> Epoch: 0 reconst loss: 0.0005278
====> Epoch: 0 kl loss: 13.9798889
====> Epoch: 0 elbo loss: 0.0005105
====> Epoch: 0 reconst loss: 0.0005028
====> Epoch: 0 kl loss: 15.4382496
time loss 0:13
====> Epoch: 10 elbo loss: 0.0000859
====> Epoch: 10 reconst loss: 0.0000753
====> Epoch: 10 kl loss: 21.2945156
====> Epoch: 10 elbo loss: 0.0001066
====> Epoch: 10 reconst loss: 0.0000949
====> Epoch: 10 kl loss: 23.5161953
time loss 2:46
====> Epoch: 20 elbo loss: 0.0000799
====> Epoch: 20 reconst loss: 0.0000679
====> Epoch: 20 kl loss: 24.0891991
====> Epoch: 20 elbo loss: 0.0000783
====> Epoch: 20 reconst loss: 0.0000653
====> Epoch: 20 kl loss: 25.9985199
time loss 5:21


In [ ]:
device = 'cpu'
model = torch.load('./vae_model1.pth', map_location=device)

In [ ]:
latent_dim = 20

In [ ]:
import torchvision
from torchvision.utils import save_image
# Generation
w = 0
import random
with torch.no_grad():    
    
    noise = torch.randn(100, latent_dim).to(device)
    c = [i for i in range(10) for _ in range(10)]
    c = one_hot_encode(c, num_classes = num_class)
    c = torch.tensor(c).to(device)
    generated_imgs = model.decoder(noise, c)
    resized_image = torchvision.transforms.Resize((50, 50))(generated_imgs)
    save_image(resized_image, './pictures/genera1.png', nrow=10)

In [ ]:
device = 'cuda:0'
model = torch.load('./vae_model1.pth', map_location=device)

In [ ]:
latent_dim = 20

In [ ]:
model.eval()
with torch.no_grad():
    num = 73200
    t = 100
    num0 = int(num/t)
    for i in range(t):
        noise = torch.randn(num0, latent_dim).to(device)
        label0 = torch.randint(low=0, high=10, size=(num0,), dtype=torch.int32)
        c = label0.tolist()
        c = one_hot_encode(c, num_classes = num_class)
        c = torch.tensor(c).to(device)
        x0 = model.decoder(noise, c)
        if(i == 0):
            x = x0
            label = label0
        else:
            x = torch.cat((x, x0), 0)
            label = torch.cat((label, label0), 0)

In [ ]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, x, label):
        self.x = x
        self.label = label
        self.n = x.shape[0]

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        return self.x[idx], self.label[idx]

In [ ]:
split_idx = 73200 - 26000
train_x, train_label = x[:split_idx], label[:split_idx]
test_x, test_label = x[split_idx:], label[split_idx:]
train_dataset = CustomDataset(train_x, train_label)
test_dataset = CustomDataset(test_x, test_label)

import pickle
train_file = './data/model1/train_dataset.pkl'
test_file = './data/model1/test_dataset.pkl'

with open(train_file, 'wb') as f:
    pickle.dump(train_dataset, f)
with open(test_file, 'wb') as f:
    pickle.dump(test_dataset, f)

In [ ]:
import pickle
train_file = './data/model1/train_dataset.pkl'
test_file = './data/model1/test_dataset.pkl'

with open(train_file, 'rb') as f:
    train_dataset = pickle.load(f)
with open(test_file, 'rb') as f:
    test_dataset = pickle.load(f)
    
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

print(len(train_dataset), len(test_dataset))

In [ ]:
import os
from torch.utils.data import DataLoader, Subset

num_samples = 30000
indices = np.random.choice(len(train_dataset), num_samples, replace=False)
subset_train_dataset = Subset(train_dataset, indices)

print("start deleting")
import shutil
save_dir = './samples/base1'
if os.path.exists(save_dir) and os.path.isdir(save_dir):
    shutil.rmtree(save_dir)
else:
    print(f"Directory does not exist.")
print("done deleting")

os.makedirs(save_dir, exist_ok=True)

print("start saving")
def save_images(dataset, save_dir):
    for idx, (image, label) in enumerate(dataset):
        image = transforms.ToPILImage()(image)
        image.save(os.path.join(save_dir, f'image_{idx}.png'))
save_images(subset_train_dataset, save_dir)
print("done saving")